Data sets taken from Kaggle's Football Data from Transfermarkt: https://www.kaggle.com/datasets/davidcariboo/player-scores
The dataset contains multiple CSV files with information about players, appearances, competitions, and games.

In [1]:
!pip install -q findspark
!pip install -q pyspark

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

It is required to answer the following questions:

1. the three countries with the most players born in that country. The order will be displayed in descending order.

In [22]:
players = spark.read.option('header',True).option('inferschema', True).csv('./data/players.csv')
players.show(truncate=False)
players.printSchema()

+---------+---------------+---------------------+---------------------+----------------+----------------------+-------------+----------+------------------+-----+------------+-------------------+---------------------------+---------------------------------------------------------------------------+
|player_id|current_club_id|name                 |pretty_name          |country_of_birth|country_of_citizenship|date_of_birth|position  |sub_position      |foot |height_in_cm|market_value_in_gbp|highest_market_value_in_gbp|url                                                                        |
+---------+---------------+---------------------+---------------------+----------------+----------------------+-------------+----------+------------------+-----+------------+-------------------+---------------------------+---------------------------------------------------------------------------+
|38790    |28095          |dmitri-golubov       |Dmitri Golubov       |UdSSR           |Russia         

In [23]:
from pyspark.sql.functions import col, desc
players.groupBy('country_of_birth').count().filter(players.country_of_birth.isNotNull()).orderBy(desc('count')).show(3)

+----------------+-----+
|country_of_birth|count|
+----------------+-----+
|          France| 1694|
|           Spain| 1388|
|           Italy| 1312|
+----------------+-----+
only showing top 3 rows



2. Show the players with red card. The list must show the player's name and the number of red cards for each player.

In [24]:
appareances = spark.read.option('header',True).option('inferschema', True).csv('./data/appearances.csv')
appareances.show()

+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|player_id|game_id|appearance_id|competition_id|player_club_id|goals|assists|minutes_played|yellow_cards|red_cards|
+---------+-------+-------------+--------------+--------------+-----+-------+--------------+------------+---------+
|    52453|2483937|2483937_52453|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2479929|2479929_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2483937|2483937_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2484582|2484582_67064|           RU1|         28095|    0|      0|            55|           0|        0|
|    67064|2485965|2485965_67064|           RU1|         28095|    0|      0|            90|           0|        0|
|    67064|2487345|2487345_67064|           RU1|         28095|    0|   

In [25]:
from pyspark.sql.functions import sum

players_red_cards = appareances.groupBy('player_id').agg(sum('red_cards').alias('total_red_cards'))

players_appareances = players_red_cards.join(players, players_red_cards['player_id'] == players['player_id'], 'left')

players_appareances.select(col('pretty_name'), col('total_red_cards')).filter(col('total_red_cards') > 0).orderBy(desc('total_red_cards')).show()

+----------------+---------------+
|     pretty_name|total_red_cards|
+----------------+---------------+
|      Thomas Lam|              5|
| Mario Balotelli|              5|
|    Granit Xhaka|              5|
|Domenico Berardi|              5|
|          Hilton|              5|
| Damien Da Silva|              4|
| Yannick Cahuzac|              4|
|     Jaume Costa|              4|
|   Heung Min Son|              4|
| Gabriel Paletta|              4|
|    Jordan Amavi|              4|
|      Ante Rebic|              4|
| Stefan Mitrovic|              4|
| Rodrigo De Paul|              4|
|         Rodrigo|              4|
|     Ivan Ordets|              4|
| Roberto Soriano|              4|
|      Jordi Amat|              4|
|  Ognjen Vranjes|              4|
|      Allan Nyom|              4|
+----------------+---------------+
only showing top 20 rows



3. Show the number of games played in the Premier League. The result should have the name of the league and the number of games played in it.

In [26]:
competitions = spark.read.option('header',True).option('inferschema', True).csv('./data/competitions.csv')
competitions.show()

+--------------+--------------------+------------------+----------+------------+--------------------+-------------+--------------------+
|competition_id|                name|              type|country_id|country_name|domestic_league_code|confederation|                 url|
+--------------+--------------------+------------------+----------+------------+--------------------+-------------+--------------------+
|            L1|          bundesliga|        first_tier|        40|     Germany|                  L1|       europa|https://www.trans...|
|           DFB|           dfb-pokal|      domestic_cup|        40|     Germany|                  L1|       europa|https://www.trans...|
|           DFL|        dfl-supercup|domestic_super_cup|        40|     Germany|                  L1|       europa|https://www.trans...|
|           NL1|          eredivisie|        first_tier|       122| Netherlands|                 NL1|       europa|https://www.trans...|
|           NLP|     toto-knvb-beker|    

In [19]:
games = spark.read.option('header',True).option('inferschema', True).csv('./data/games.csv')
games.show()

+-------+----------------+------+-------------+----------+------------+------------+---------------+---------------+------------------+------------------+--------------------+----------+--------------------+--------------------+
|game_id|competition_code|season|        round|      date|home_club_id|away_club_id|home_club_goals|away_club_goals|home_club_position|away_club_position|             stadium|attendance|             referee|                 url|
+-------+----------------+------+-------------+----------+------------+------------+---------------+---------------+------------------+------------------+--------------------+----------+--------------------+--------------------+
|2457642|            NLSC|  2014|        Final|2014-08-03|        1269|         610|              1|              0|              null|              null| Johan Cruijff ArenA|     42000|      Danny Makkelie|https://www.trans...|
|2639088|            BESC|  2013|        Final|2014-07-20|          58|         498|

In [20]:
from pyspark.sql.functions import count
games_premier_liga = games.join(competitions, col('competition_code') == col('competition_id'), 'left_outer')
games_premier_liga.groupBy('name').count().filter(col('name') == 'premier-league').show()

+--------------+-----+
|          name|count|
+--------------+-----+
|premier-league| 2809|
+--------------+-----+



4. Show three leagues with the highest attendance by taking into account all the games played in them. The result must contain: league name and total attendance and must appear in descending order.

In [21]:
from pyspark.sql.functions import sum

games_attendance = games_premier_liga.groupBy('name').agg(
    sum('attendance').alias('att')
).orderBy(desc('att')).show(3)

+--------------+--------+
|          name|     att|
+--------------+--------+
|premier-league|86964852|
|    bundesliga|78102473|
|        laliga|62943533|
+--------------+--------+
only showing top 3 rows

